# **SECTION 1**
## Data Prepration

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("Dataframe")
         .getOrCreate())

## Reading the dataset from drive

In [ ]:
## read the dataset 

ipl_venue = spark.read.option('header','true').csv('/content/drive/MyDrive/ipl_venue.csv')
ipl_matches = spark.read.option('header','true').csv('/content/drive/MyDrive/ipl_matches.csv')
ipl_ball = spark.read.option('header','true').csv('/content/drive/MyDrive/ipl_ball_by_ball.csv')

ipl_venue.show()

+--------+--------------------+--------------+
|venue_id|               venue|          city|
+--------+--------------------+--------------+
|       1|Dr. Y.S. Rajasekh...| Visakhapatnam|
|       2|Sharjah Cricket S...|       Sharjah|
|       3|JSCA Internationa...|        Ranchi|
|       4|Saurashtra Cricke...|        Rajkot|
|       5|Shaheed Veer Nara...|        Raipur|
|       6|Maharashtra Crick...|          Pune|
|       7|Subrata Roy Sahar...|          Pune|
|       8|    St George's Park|Port Elizabeth|
|       9|Vidarbha Cricket ...|        Nagpur|
|      12|   Brabourne Stadium|        Mumbai|
|      13|Dr DY Patil Sport...|        Mumbai|
|      14|    Wankhede Stadium|        Mumbai|
|      15|        Eden Gardens|       Kolkata|
|      16|       Nehru Stadium|         Kochi|
|      17|De Beers Diamond ...|     Kimberley|
|      18|          Green Park|        Kanpur|
|      19|New Wanderers Sta...|  Johannesburg|
|      20|Sawai Mansingh St...|        Jaipur|
|      21|Hol

## Looking for the data types of the coloumns

In [ ]:
## Looking for the data types of the coloumns

ipl_venue.printSchema()
ipl_matches.printSchema()
ipl_ball.printSchema()

root
 |-- venue_id: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- match_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue_id: string (nullable = true)
 |-- neutral_venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)

root
 |-- match_id: string (nullable = true)
 |-- inning: string (nullable = true)
 |-- overs: string (nullable = true)
 |-- ball: string (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nulla

## Converting String to numeric

In [ ]:
#Converting String to numeric

from pyspark.sql.types import IntegerType

ipl_matches = ipl_matches.withColumn("match_id", ipl_matches["match_id"].cast(IntegerType()))
ipl_matches = ipl_matches.withColumn("result_margin", ipl_matches["result_margin"].cast(IntegerType()))
ipl_venue = ipl_venue.withColumn("venue_id", ipl_venue["venue_id"].cast(IntegerType()))

ipl_ball = ipl_ball.withColumn("inning", ipl_ball["inning"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("overs", ipl_ball["overs"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("batsman_runs", ipl_ball["batsman_runs"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("extra_runs", ipl_ball["extra_runs"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("total_runs", ipl_ball["total_runs"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("non_boundary", ipl_ball["overs"].cast(IntegerType()))
ipl_ball = ipl_ball.withColumn("is_wicket", ipl_ball["is_wicket"].cast(IntegerType()))


ipl_matches.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue_id: string (nullable = true)
 |-- neutral_venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: integer (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



# **SECTION 2**
## 1. Top 3 venues which hosted the most number of eliminator matches

In [ ]:
from pyspark.sql import SQLContext, Row

#Checking distinct values and value counts of eliminator
ipl_matches.groupBy("eliminator").count().show()

## Joining ipl_venue and ipl_match as ipl_vm. (vm = venue match)
ipl_vm = ipl_venue.join(ipl_matches, ipl_venue.venue_id == ipl_matches.venue_id).drop(ipl_matches.venue_id)

df1 = ipl_vm.groupBy("venue","eliminator").count().sort("count",ascending=False).filter(ipl_vm.eliminator=="Y").show()

+----------+-----+
|eliminator|count|
+----------+-----+
|        NA|    4|
|         Y|   13|
|         N|  799|
+----------+-----+

+--------------------+----------+-----+
|               venue|eliminator|count|
+--------------------+----------+-----+
|Dubai Internation...|         Y|    3|
|Sheikh Zayed Stadium|         Y|    2|
|            Newlands|         Y|    1|
|    Wankhede Stadium|         Y|    1|
|    Feroz Shah Kotla|         Y|    1|
|Saurashtra Cricke...|         Y|    1|
|M.Chinnaswamy Sta...|         Y|    1|
|MA Chidambaram St...|         Y|    1|
|Rajiv Gandhi Inte...|         Y|    1|
|Sardar Patel Stad...|         Y|    1|
+--------------------+----------+-----+



## 2. Most number of catches taken by a player in IPL history

In [ ]:
ipl_ball.groupBy("bowler","dismissal_kind").count().filter(ipl_ball.dismissal_kind == "caught").show()

+---------------+--------------+-----+
|         bowler|dismissal_kind|count|
+---------------+--------------+-----+
|        S Gopal|        caught|   30|
|       ND Doshi|        caught|    1|
|      GH Vihari|        caught|    1|
|       TL Suman|        caught|    6|
|SMSM Senanayake|        caught|    1|
|        RR Raje|        caught|    4|
|        D Wiese|        caught|   13|
|  Parvez Rasool|        caught|    3|
|     DL Vettori|        caught|   15|
|       S Badree|        caught|    6|
|     M de Lange|        caught|    3|
|      CH Morris|        caught|   53|
|       AR Patel|        caught|   44|
|       S Dhawan|        caught|    3|
|       MA Khote|        caught|    2|
| PD Collingwood|        caught|    3|
|      DNT Zoysa|        caught|    1|
|   MC Henriques|        caught|   32|
|     TM Dilshan|        caught|    3|
|J Syed Mohammad|        caught|    6|
+---------------+--------------+-----+
only showing top 20 rows



## 3. Report for highest wicket taker in matches which were affected by Duckworth-Lewis’s method (D/L method).

In [ ]:
## Joining ipl_match and ipl_ball as ipl_mb. (mb = match ball)
ipl_mb = ipl_matches.join(ipl_ball,ipl_matches.match_id == ipl_ball.match_id)

ipl_mb.select("bowler","method","is_wicket").show()
ipl_mb.select("is_wicket").distinct().show()

df_mb = ipl_mb.groupBy("bowler","method","is_wicket").count()
df2 = df_mb.filter((ipl_mb.is_wicket == "1") & (ipl_mb.method == "D/L")).sort("count",ascending=False)
print("No.of bowlers with wicket inD/L: ",df2.count())
df2.show()



+------------+------+---------+
|      bowler|method|is_wicket|
+------------+------+---------+
|CRD Fernando|    NA|        0|
|CRD Fernando|    NA|        0|
|CRD Fernando|    NA|        0|
|   R McLaren|    NA|        0|
|   R McLaren|    NA|        0|
|   R McLaren|    NA|        0|
|   R McLaren|    NA|        0|
|   R McLaren|    NA|        0|
|   R McLaren|    NA|        0|
|  KA Pollard|    NA|        0|
|  KA Pollard|    NA|        1|
|  KA Pollard|    NA|        0|
|  KA Pollard|    NA|        0|
|  KA Pollard|    NA|        0|
|  KA Pollard|    NA|        0|
|  KA Pollard|    NA|        1|
|   R McLaren|    NA|        0|
|    DW Steyn|    NA|        0|
|    DW Steyn|    NA|        0|
|    DW Steyn|    NA|        0|
+------------+------+---------+
only showing top 20 rows

+---------+
|is_wicket|
+---------+
|        1|
|        0|
+---------+

No.of bowlers with wicket inD/L:  72
+----------------+------+---------+-----+
|          bowler|method|is_wicket|count|
+-----------

## 4. Return a report for highest strike rate by a batsman in non powerplay overs(7-20 overs)

In [ ]:
legal=ipl_vmb[(~ipl_vmb['extras_type'].isin(['wides', 'noballs'])) & (ipl_vmb['overs'].between(7,20))]
#filtering out wides and no balls, and taking  only powerplay overs(7-20) 

runs=legal.groupBy('batsman').sum('batsman_runs')
#finding out total runs by each batsman

ball=legal.groupby('batsman').count()
#finding out total balls faced by each batsman

balls=ball.withColumnRenamed("batsman","batsman_1")

joined=balls.join(runs, balls.batsman_1 == runs.batsman, 'inner')
#In pandas we could  just do strike_rate=runs*100/balls and we could get strike rate. We can't do same in pyspark , so first I will join the two result set and then find the strike rate

result_4=joined.withColumn("Strike rate", col("sum(batsman_runs)")*100/col("count")).sort('Strike rate',ascending=False)
#Dividing the two columns and getting the strike rate the hard way.

result_4.select('batsman','Strike rate').show(10)
#Top 10 batsman with highest strike in powerplay


+------------+------------------+
|     batsman|       Strike rate|
+------------+------------------+
|  B Stanlake|             250.0|
|Kamran Akmal|213.88888888888889|
|  ER Dwivedi|211.11111111111111|
|    Umar Gul|205.26315789473685|
|    RS Sodhi|             200.0|
| AC Blizzard|             200.0|
|    W Jaffer|             188.0|
|   DJM Short|187.09677419354838|
|   LJ Wright|             186.0|
|  AD Russell|181.85185185185185|
+------------+------------------+
only showing top 10 rows



## 5. Return a report for highest extra runs in a venue (stadium, city)

In [ ]:
## Joing ipl_vm and ipl_ball as ipl_mvb. (vmb = venue match ball)
ipl_vmb = ipl_vm.join(ipl_ball, ipl_vm.match_id == ipl_ball.match_id).drop(ipl_vm.match_id)
#ipl_vmb.printSchema()

ipl_vmb.groupBy("venue","city").sum("extra_runs").sort("sum(extra_runs)",ascending=False).show()

+--------------------+----------+---------------+
|               venue|      city|sum(extra_runs)|
+--------------------+----------+---------------+
|M.Chinnaswamy Sta...| Bengaluru|           1355|
|    Wankhede Stadium|    Mumbai|           1232|
|        Eden Gardens|   Kolkata|           1215|
|    Feroz Shah Kotla|     Delhi|           1114|
|MA Chidambaram St...|   Chennai|            905|
|Rajiv Gandhi Inte...| Hyderabad|            900|
|Sawai Mansingh St...|    Jaipur|            791|
|Punjab Cricket As...|Chandigarh|            566|
|Dubai Internation...|     Dubai|            492|
|Sheikh Zayed Stadium| Abu Dhabi|            387|
|Dr DY Patil Sport...|    Mumbai|            320|
|Maharashtra Crick...|      Pune|            315|
|Sharjah Cricket S...|   Sharjah|            287|
|           Kingsmead|    Durban|            277|
|Punjab Cricket As...|Chandigarh|            271|
|   Brabourne Stadium|    Mumbai|            246|
|Subrata Roy Sahar...|      Pune|            237|


## 6. Return a report for the cricketers with the most number of players of the match award in neutral venues.

In [ ]:
df_vmb = ipl_vmb.groupBy("player_of_match","neutral_venue").count()
#returning counts of players in neutral venue

df3 = df_vmb.filter(df_vmb.neutral_venue == "1").sort("count")
# Filters out the players awarded with neutral venue

print("Total no.of player_of_match in neutral_venue:", df3.count())
df3.show()


Total no.of player_of_match in neutral_venue: 55
+---------------+-------------+-----+
|player_of_match|neutral_venue|count|
+---------------+-------------+-----+
|     DL Vettori|            1|  108|
|       PV Tambe|            1|  171|
|       CH Gayle|            1|  187|
|       RP Singh|            1|  199|
|       MS Dhoni|            1|  215|
|        A Singh|            1|  218|
|       R Dravid|            1|  221|
|       PA Patel|            1|  230|
|      YS Chahal|            1|  230|
|       A Mishra|            1|  234|
|Harbhajan Singh|            1|  234|
|      MM Sharma|            1|  237|
|        M Vijay|            1|  238|
|    S Badrinath|            1|  238|
|      G Gambhir|            1|  240|
|       V Sehwag|            1|  241|
|      RS Bopara|            1|  242|
|      LR Shukla|            1|  242|
|      DP Nannes|            1|  243|
|      AM Rahane|            1|  244|
+---------------+-------------+-----+
only showing top 20 rows



## 7. Top 10 players with the highest batting average

In [ ]:
from pyspark.sql.functions import col

player_dismissed=ipl_vmb.groupby('player_dismissed').count()
#Counting number of times a player is dismissed

total_runs=ipl_vmb.groupBy('batsman').sum('batsman_runs')
joined_data=player_dismissed.join(total_runs, player_dismissed.player_dismissed == total_runs.batsman, 'inner')

result_7=joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)
#Dividing runs by number of times they have been dismissed

result_7.select('batsman','Batsmen Avg').show(10)

## 8. out who has officiated (as an umpire) the most number of matches in IPL.

In [ ]:
#ipl_vmb.printSchema()

df4 = ipl_vmb.select("match_id","umpire1")
df5 = ipl_vmb.select("match_id","umpire2")
df6 = df4.union(df5)

df_umpires = df6.dropDuplicates(["match_id","umpire1"])

df_7 = df_umpires.groupBy("umpire1").count().sort("count", ascending = False)
print("Total no.of umpires ",df_7.count())
df_7.show()



Total no.of umpires  56
+--------------------+-----+
|             umpire1|count|
+--------------------+-----+
|              S Ravi|  121|
|     HDPK Dharmasena|   94|
|        AK Chaudhary|   87|
|       C Shamshuddin|   82|
|           M Erasmus|   65|
|         Nitin Menon|   57|
|           CK Nandan|   57|
|          SJA Taufel|   55|
|           Asad Rauf|   51|
|         VA Kulkarni|   50|
|        BNJ Oxenford|   48|
|         CB Gaffaney|   47|
|           RJ Tucker|   46|
|         BR Doctrove|   42|
|         RE Koertzen|   41|
|           Aleem Dar|   38|
|           BF Bowden|   37|
|            NJ Llong|   37|
|KN Ananthapadmana...|   33|
|      RK Illingworth|   31|
+--------------------+-----+
only showing top 20 rows



## 9. Venue details of the match where V Kohli scored his highest individual runs in IPL.

In [ ]:
## Venue details of the match where V Kohli scored his highest individual runs in IPL.

df_virat = ipl_vmb.select("match_id","venue","batsman","batsman_runs").filter(ipl_vmb.batsman == "V Kohli")
df8 = df_virat.groupBy("match_id","batsman","venue").sum("batsman_runs").sort("sum(batsman_runs)", ascending = False)
print("Total Matches played by Kholi: ",df8.count())
df8.show()

Total Matches played by Kholi:  184
+--------+-------+--------------------+-----------------+
|match_id|batsman|               venue|sum(batsman_runs)|
+--------+-------+--------------------+-----------------+
|  980999|V Kohli|M.Chinnaswamy Sta...|              113|
|  980987|V Kohli|M.Chinnaswamy Sta...|              109|
|  980969|V Kohli|M.Chinnaswamy Sta...|              108|
|  980937|V Kohli|Saurashtra Cricke...|              100|
| 1178410|V Kohli|        Eden Gardens|              100|
|  598054|V Kohli|    Feroz Shah Kotla|               99|
|  598048|V Kohli|M.Chinnaswamy Sta...|               93|
| 1136574|V Kohli|    Wankhede Stadium|               92|
| 1216525|V Kohli|Dubai Internation...|               90|
| 1175372|V Kohli|M.Chinnaswamy Sta...|               84|
|  829795|V Kohli|    Wankhede Stadium|               82|
|  980931|V Kohli|Maharashtra Crick...|               80|
|  980921|V Kohli|M.Chinnaswamy Sta...|               79|
|  980995|V Kohli|        Eden Garde

## 10. Creative Case study:
### Analyze how winning/losing tosses can impact a match and it's result?

In [ ]:
ipl_vmb.printSchema()
ipl_vmb.select("toss_winner","toss_decision","result","result_margin","winner").show()

root
 |-- venue_id: integer (nullable = true)
 |-- venue: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- neutral_venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: integer (nullable = true)
 |-- eliminator: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- inning: integer (nullable = true)
 |-- overs: integer (nullable = true)
 |-- ball: string (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- batsman_runs: integer (nullable =

In [ ]:
df9 = ipl_vmb.groupBy("toss_winner","winner","toss_decision","result").sum("result_margin")
df9.sort("toss_winner").show()

+-------------------+--------------------+-------------+-------+------------------+
|        toss_winner|              winner|toss_decision| result|sum(result_margin)|
+-------------------+--------------------+-------------+-------+------------------+
|Chennai Super Kings|Royal Challengers...|          bat|wickets|              1705|
|Chennai Super Kings|Kolkata Knight Ri...|          bat|   runs|              1860|
|Chennai Super Kings|    Delhi Daredevils|          bat|wickets|              4724|
|Chennai Super Kings|     Kings XI Punjab|        field|    tie|              null|
|Chennai Super Kings|     Deccan Chargers|          bat|wickets|              1464|
|Chennai Super Kings| Sunrisers Hyderabad|        field|   runs|              5456|
|Chennai Super Kings|     Kings XI Punjab|        field|   runs|             18080|
|Chennai Super Kings|       Pune Warriors|          bat|wickets|              1701|
|Chennai Super Kings|    Rajasthan Royals|        field|   runs|            

# **SECTION 3**

In [ ]:
import sqlite3

connection = sqlite3.connect("ipl.db")

c = connection.cursor()

def create_table():
  c.execute("CREATE TABLE IF NOT EXISTS IPL_DATA()")

def data_entry():
  c.execute("INSERT INTO IPL_DATA VALUES()")
  connection.commit()
  c.close()
  connection.close()

create_table()
data_entry()  

